In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/My\ Drive/

 algo  'สำเนาของ 3-1 Final DQA Tools v.0.6_forPrint.xlsx'


In [3]:
!pip install ta

In [4]:

import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
import ta
from ta import add_all_ta_features
from ta.utils import dropna
template = "plotly_dark"

In [5]:
# Manual Download https://forexsb.com/historical-forex-data -> no volume

In [29]:
import hashlib
import random

def color_from_time(time_value):
    """
    Generate a consistent random HEX color string based on a datetime or string.
    """
    if pd.isna(time_value):
        return "#cccccc"  # default gray for NaT

    # Convert datetime to string and hash it
    time_str = str(time_value)
    seed = int(hashlib.md5(time_str.encode()).hexdigest(), 16)

    # Seed RNG and generate color
    random.seed(seed)
    return '#{:02X}{:02X}{:02X}'.format(
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255)
    )

In [6]:
profit_color_continuous_scale =["red","orange" ,"White","lightgreen","green"]

In [7]:
## Historical Order Data
raw_df = pd.read_excel('/content/drive/MyDrive/algo/input/ReportHistory-32012857.xlsx',skiprows=6)
raw_df = raw_df[~raw_df['Commission'].isnull()]
time_config = {
     "format":'%Y.%m.%d %H:%M:%S',
     "errors":'coerce'
}
# raw_df['Time.1'] = pd.to_datetime(raw_df['Time.1'], format='%Y.%m.%d %H:%M:%S', errors='coerce')
raw_df["Open Time"] = pd.to_datetime(raw_df["Time"],**time_config)
raw_df["Close Time"] = pd.to_datetime(raw_df["Time.1"],**time_config)
raw_df = raw_df.dropna(subset=['Time.1'])
raw_df["Volume"] = pd.to_numeric(raw_df["Volume"], errors="coerce")


raw_df = raw_df[raw_df['Open Time']>='2025-04-13']
raw_df = raw_df[raw_df['Close Time']<'2025-04-17']

# Step 2: Convert to UTC
# raw_df["Open Time"] = raw_df["Open Time"].dt.tz_localize("Asia/Bangkok")
# raw_df["Close Time"] = raw_df["Close Time"].dt.tz_localize("Asia/Bangkok")
# raw_df["Open Time"] = raw_df["Open Time"].dt.tz_convert("UTC")
# raw_df["Close Time"] = raw_df["Close Time"].dt.tz_convert("UTC")
raw_df["Open Time"] = raw_df["Open Time"] - pd.Timedelta(hours=3)
raw_df["Close Time"] = raw_df["Close Time"] - pd.Timedelta(hours=3)

raw_df["Open Time (15m)"] = raw_df["Open Time"].dt.floor("15min")
order_counts = raw_df.groupby("Open Time (15m)").size().reset_index(name="Orders in 15m")
raw_df = raw_df.merge(order_counts, on="Open Time (15m)", how="left")
# Drop rows where Volume is NaN (invalid)
raw_df = raw_df[raw_df["Volume"].notna()]
raw_df = raw_df.rename(columns={
    "Price": "Open Price",
    "Price.1": "Close Price",
    "Profit": "Profit USD"
})
dtype_mapping = {

    "Profit USD":'int64'
}
raw_df = raw_df.astype(dtype_mapping)
# Drop irrelevant or unnamed columns
df = raw_df[["Open Time (15m)","Orders in 15m","Open Time", "Close Time", "Symbol", "Type", "Volume", "Open Price", "Close Price", "T / P", "Profit USD"]]


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [8]:
df

,Open Time (15m),Orders in 15m,Open Time,Close Time,Symbol,Type,Volume,Open Price,Close Price,T / P,Profit USD
0,2025-04-14 02:00:00,2,2025-04-14 02:04:18,2025-04-14 02:04:59,XAUUSDc,buy,2.0,3243.22,3243.23,3246.22,2
1,2025-04-14 02:00:00,2,2025-04-14 02:04:59,2025-04-14 05:55:13,XAUUSDc,buy,2.0,3243.75,3234.39,3246.75,-1872
2,2025-04-14 02:15:00,3,2025-04-14 02:16:35,2025-04-14 05:55:13,XAUUSDc,buy,2.0,3240.79,3234.26,3243.79,-1306
3,2025-04-14 02:15:00,3,2025-04-14 02:18:55,2025-04-14 05:55:12,XAUUSDc,buy,2.0,3237.56,3234.26,3240.56,-660
4,2025-04-14 02:15:00,3,2025-04-14 02:25:09,2025-04-14 05:55:12,XAUUSDc,buy,2.0,3234.49,3234.03,3237.49,-92
...,...,...,...,...,...,...,...,...,...,...,...
443,2025-04-16 16:15:00,5,2025-04-16 16:18:28,2025-04-16 16:19:33,XAUUSDc,buy,1.0,3328.74,3328.76,3331.74,2
444,2025-04-16 16:15:00,5,2025-04-16 16:19:34,2025-04-16 16:26:51,XAUUSDc,buy,1.0,3329.15,3332.27,3332.15,312
445,2025-04-16 16:15:00,5,2025-04-16 16:24:24,2025-04-16 16:26:51,XAUUSDc,buy,1.0,3332.23,3332.28,3335.23,5
446,2025-04-16 16:15:00,5,2025-04-16 16:26:51,2025-04-16 16:27:20,XAUUSDc,buy,1.0,3332.69,3332.73,3335.69,4


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Open Time (15m)  448 non-null    datetime64[ns]
 1   Orders in 15m    448 non-null    int64         
 2   Open Time        448 non-null    datetime64[ns]
 3   Close Time       448 non-null    datetime64[ns]
 4   Symbol           448 non-null    object        
 5   Type             448 non-null    object        
 6   Volume           448 non-null    float64       
 7   Open Price       448 non-null    object        
 8   Close Price      448 non-null    object        
 9   T / P            447 non-null    object        
 10  Profit USD       448 non-null    int64         
dtypes: datetime64[ns](3), float64(1), int64(2), object(5)
memory usage: 38.6+ KB


In [10]:
import plotly.express as px


In [11]:

# Example DataFrame cleanup if not already done
df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
df = df[df["Volume"].notna()]

# Melt the dataframe to show both Open and Close price as individual points
df_melted = df.melt(
    id_vars=["Open Time (15m)","Open Time", "Close Time", "Type", "Profit USD"],
    value_vars=["Open Price", "Close Price"],
    var_name="Price Type",
    value_name="Price"
)

# Use Open Time for both points to align them visually
df_melted["Time"] = df_melted.apply(
    lambda row: row["Open Time"] if row["Price Type"] == "Open Price" else row["Close Time"],
    axis=1
)

# Color profit: positive = green, negative = red
df_melted["Profit Color"] = df_melted["Profit USD"].apply(
    lambda x: "green" if x >= 0 else "red")
df_melted["Profit Size"] = df_melted["Profit USD"].apply(
    lambda x: abs(x))

fig = px.scatter(
    df_melted,
    x="Time",
    y="Price",
    color="Profit USD",
    # color_continuous_scale='YlGn',
    color_continuous_midpoint=0,  # This centers the color scale at 0
    range_color=[-300, 300],           # Set min/max of color range
    # size="Profit Size",
    color_continuous_scale=profit_color_continuous_scale,
    symbol="Type",
    facet_col="Type",
    title="Entry and Exit Prices by Trade",
    hover_data=["Profit USD", "Type"]
)

fig.update_layout(height=500, xaxis_title="Time", yaxis_title="Price",template=template)
fig.show()

<ipython-input-11-ebc2b8d15928>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")


In [12]:
df_melted

,Open Time (15m),Open Time,Close Time,Type,Profit USD,Price Type,Price,Time,Profit Color,Profit Size
0,2025-04-14 02:00:00,2025-04-14 02:04:18,2025-04-14 02:04:59,buy,2,Open Price,3243.22,2025-04-14 02:04:18,green,2
1,2025-04-14 02:00:00,2025-04-14 02:04:59,2025-04-14 05:55:13,buy,-1872,Open Price,3243.75,2025-04-14 02:04:59,red,1872
2,2025-04-14 02:15:00,2025-04-14 02:16:35,2025-04-14 05:55:13,buy,-1306,Open Price,3240.79,2025-04-14 02:16:35,red,1306
3,2025-04-14 02:15:00,2025-04-14 02:18:55,2025-04-14 05:55:12,buy,-660,Open Price,3237.56,2025-04-14 02:18:55,red,660
4,2025-04-14 02:15:00,2025-04-14 02:25:09,2025-04-14 05:55:12,buy,-92,Open Price,3234.49,2025-04-14 02:25:09,red,92
...,...,...,...,...,...,...,...,...,...,...
891,2025-04-16 16:15:00,2025-04-16 16:18:28,2025-04-16 16:19:33,buy,2,Close Price,3328.76,2025-04-16 16:19:33,green,2
892,2025-04-16 16:15:00,2025-04-16 16:19:34,2025-04-16 16:26:51,buy,312,Close Price,3332.27,2025-04-16 16:26:51,green,312
893,2025-04-16 16:15:00,2025-04-16 16:24:24,2025-04-16 16:26:51,buy,5,Close Price,3332.28,2025-04-16 16:26:51,green,5
894,2025-04-16 16:15:00,2025-04-16 16:26:51,2025-04-16 16:27:20,buy,4,Close Price,3332.73,2025-04-16 16:27:20,green,4


In [25]:
order_count_by_close = df_melted.groupby("Close Time").size().reset_index(name="Order Count")
order_count_by_close.sort_values('Order Count')

,Close Time,Order Count
24,2025-04-14 10:42:50,2
11,2025-04-14 06:29:14,2
12,2025-04-14 06:29:15,2
13,2025-04-14 07:24:28,2
14,2025-04-14 07:57:00,2
...,...,...
8,2025-04-14 05:55:12,6
131,2025-04-15 12:14:36,6
70,2025-04-14 16:38:02,8
56,2025-04-14 14:35:43,8


In [13]:
hist_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume']

hist_df = pd.read_csv('/content/drive/MyDrive/algo/input/XAUUSD_M5.csv',names=hist_cols)

In [14]:
hist_df

,datetime,open,high,low,close,volume
0,2023-11-29 06:05,2044.175,2044.645,2043.065,2043.455,5
1,2023-11-29 06:10,2043.505,2044.205,2043.225,2044.035,5
2,2023-11-29 06:15,2044.125,2045.025,2043.835,2044.535,5
3,2023-11-29 06:20,2044.535,2045.355,2044.375,2045.315,5
4,2023-11-29 06:25,2045.235,2045.325,2044.284,2045.085,5
...,...,...,...,...,...,...
99995,2025-04-30 03:35,3307.045,3307.165,3305.325,3306.735,5
99996,2025-04-30 03:40,3306.535,3307.255,3303.335,3306.875,5
99997,2025-04-30 03:45,3306.455,3308.715,3306.325,3307.585,5
99998,2025-04-30 03:50,3307.615,3307.895,3306.595,3307.405,5


In [15]:

# Step 2: Prepare datetime
hist_df['datetime'] = pd.to_datetime(hist_df['datetime'])

hist_df = hist_df[hist_df['datetime']>=df_melted['Time'].min()]
hist_df = hist_df[hist_df['datetime']<df_melted['Time'].max()]

# Step 3: Plotting
fig = go.Figure(data=[go.Candlestick(
    x=hist_df['datetime'],
    open=hist_df['open'],
    high=hist_df['high'],
    low=hist_df['low'],
    close=hist_df['close'],
    increasing_line_color='green',
    decreasing_line_color='red'
)])

fig.update_layout(
    title='Gold Spot Price (15min intervals)',
    xaxis_title='Time',
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False,
    template=template
)

fig.show()

In [16]:
ta_df = dropna(hist_df)
ta_df['datetime'] = pd.to_datetime(ta_df['datetime'], errors='coerce')
ta_df = ta_df.dropna(subset=['datetime'])

# Add ta features filling NaN values
ta_df = add_all_ta_features(
    ta_df, open="open", high="high", low="low", close="close", volume="volume", fillna=True)

In [17]:
for period in [10, 20, 50, 100, 200]:
    ta_df[f'ema_{period}'] = ta.trend.EMAIndicator(close=ta_df['close'], window=period, fillna=True).ema_indicator()

# Add RSI for common periods
for period in [14, 21]:
    ta_df[f'rsi_{period}'] = ta.momentum.RSIIndicator(close=ta_df['close'], window=period, fillna=True).rsi()

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from ta.volatility import BollingerBands
# Create subplots
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=[0.7, 0.3],
    subplot_titles=("Price with EMAs (OHLC) and Trades", "RSI")
)

# --- OHLC Chart ---
fig.add_trace(go.Candlestick(
    x=ta_df['datetime'],
    open=ta_df['open'],
    high=ta_df['high'],
    low=ta_df['low'],
    close=ta_df['close'],
    # increasing_line_color='green',
    # decreasing_line_color='red',

), row=1, col=1)
fig.update_layout(xaxis_rangeslider_visible=False)
# --- EMA Lines ---
for period in [50, 200]:
    fig.add_trace(go.Scatter(
        x=ta_df['datetime'],
        y=ta_df[f'ema_{period}'],
        mode='lines',
        name=f'EMA {period}',
        connectgaps=False,
    ), row=1, col=1)

# --- Custom Trade Markers: "buy" and "sell" ---
for trade_type, symbol in [("buy", "arrow-up"), ("sell", "arrow-down")]:
    trade_subset = df_melted[df_melted["Type"].str.lower() == trade_type]
    fig.add_trace(go.Scatter(
        x=trade_subset["Time"],
        y=trade_subset["Price"],
        mode="markers",
        name=f"{trade_type.title()} Highlight",
        marker=dict(
            symbol=symbol,
            size=16,  # Slightly bigger than main
            color=trade_subset["Close Time"].apply(color_from_time),  # Color by Close Time
            # opacity=0.3,
            line=dict(width=0)  # No border
        ),
        showlegend=False,  # Hide from legend
        hoverinfo='skip'   # No hover for background layer
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=trade_subset["Time"],
                # x=trade_subset["Open Time (15m)"],
        y=trade_subset["Price"],
        mode="markers",
        name=f"{trade_type.title()} Trade",
        connectgaps=False,
        marker=dict(
            symbol=symbol,
            size=12,
            color=trade_subset["Profit USD"],  # Color by profit
            colorscale=profit_color_continuous_scale,  # Optional custom scale
            cmin=-300,  # Set these based on your profit range
            cmax=300,
            colorbar=dict(title="Profit USD"),
            line=dict(width=1, color='black')
        ),
        hovertemplate=(
            "Time: %{x}<br>" +
            "Price: %{y}<br>" +
            "Profit USD: %{customdata[0]}<br>" +
            "Type: %{customdata[1]}"
        ),
        customdata=trade_subset[["Profit USD", "Type"]].values
    ), row=1, col=1)


# --- RSI Line ---
fig.add_trace(go.Scatter(
        x=ta_df['datetime'],
    y=ta_df['rsi_14'],
    mode='lines',
    name='RSI 14',
    line=dict(color='orange'),
    connectgaps=False,
), row=2, col=1)

# # RSI Overbought/Oversold Lines
start_date = ta_df['datetime'].iloc[0]
end_date = ta_df['datetime'].iloc[-1]

fig.add_shape(type="line", x0=start_date, x1=end_date, y0=70, y1=70,
              line=dict(dash='dash', color='red'), row=2, col=1)
fig.add_shape(type="line", x0=start_date, x1=end_date, y0=30, y1=30,
              line=dict(dash='dash', color='green'), row=2, col=1)
overbought = ta_df[ta_df['rsi_14'] > 70]
oversold = ta_df[ta_df['rsi_14'] < 30]


# bb_indicator = BollingerBands(close=ta_df["close"], window=20, window_dev=2)

# # Add bands to the DataFrame
# ta_df["bb_upper"] = bb_indicator.bollinger_hband()
# ta_df["bb_middle"] = bb_indicator.bollinger_mavg()
# ta_df["bb_lower"] = bb_indicator.bollinger_lband()
# # --- Bollinger Bands ---
# fig.add_trace(go.Scatter(
#     x=ta_df['datetime'],
#     y=ta_df['bb_upper'],
#     mode='lines',
#     name='BB Upper',
#     line=dict(color='lightblue', width=1),
#     opacity=0.6,
#     connectgaps=False
# ), row=1, col=1)

# fig.add_trace(go.Scatter(
#     x=ta_df['datetime'],
#     y=ta_df['bb_lower'],
#     mode='lines',
#     name='BB Lower',
#     line=dict(color='lightblue', width=1),
#     opacity=0.6,
#     fill='tonexty',  # Fill between upper and lower
#     connectgaps=False
# ), row=1, col=1)

# fig.add_trace(go.Scatter(
#     x=ta_df['datetime'],
#     y=ta_df['bb_middle'],
#     mode='lines',
#     name='BB Middle',
#     line=dict(color='blue', width=1),
#     connectgaps=False
# ), row=1, col=1)
# # --- Bollinger Bands ---

# Shade overbought regions on main chart
for dt in overbought['datetime']:
    fig.add_vrect(
        x0=dt, x1=dt,
        fillcolor="red", opacity=0.2,
        layer="below", line_width=0,
        row=1, col=1
    )

# Shade oversold regions on main chart
for dt in oversold['datetime']:
    fig.add_vrect(
        x0=dt, x1=dt,
        fillcolor="green", opacity=0.2,
        layer="below", line_width=0,
        row=1, col=1
    )
# Layout Settings
fig.update_layout(
    # title='Stock OHLC with EMA, Trade Entries (Buy/Sell), and RSI',
    height=800,
    legend=dict(x=0, y=1.1, orientation='h'),
    xaxis2_title='Date',
    yaxis1_title='Price',
    yaxis2_title='RSI',
    template=template
)



from datetime import datetime

# 1. Get all unique Close Times (as strings for dropdown labels)
unique_close_times = sorted(df_melted["Close Time"].dt.strftime("%Y-%m-%d %H:%M:%S").unique())
all_trace_count = len(fig.data)

# 2. Create dropdown buttons
buttons = []

# "All" button to show everything
buttons.append(dict(
    label="All",
    method="update",
    args=[{"visible": [True] * all_trace_count},
          {"title": "All Trades"}]
))

# Add one button per Close Time
for close_time_str in unique_close_times:
    visible = []
    for trace in fig.data:
        # Check if this trace's x matches only that Close Time
        trace_x = list(trace.x)
        if len(trace_x) == 0 or not isinstance(trace_x[0], datetime):
            visible.append(True)  # Show all non-trade traces
        else:
            # Filter only trades for this Close Time
            match = any(
                pd.to_datetime(close_time_str) == pd.to_datetime(row["Close Time"])
                for i, row in df_melted.iterrows()
                if row["Time"] in trace_x
            )
            visible.append(match)

    buttons.append(dict(
        label=close_time_str,
        method="update",
        args=[{"visible": visible},
              {"title": f"Trades for Close Time: {close_time_str}"}]
    ))

# 3. Add dropdown to layout
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            x=1.02,
            y=1.1,
            xanchor="left",
            yanchor="top",
            direction="down",
            showactive=True
        )
    ]
)


fig.show()


In [19]:
ta_df[ta_df['datetime'].isna()]

,datetime,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,others_dr,others_dlr,others_cr,ema_10,ema_20,ema_50,ema_100,ema_200,rsi_14,rsi_21


In [20]:
df_melted[df_melted['Time'].isna()]

,Open Time (15m),Open Time,Close Time,Type,Profit USD,Price Type,Price,Time,Profit Color,Profit Size
